In [ ]:
import pandas as pd
from datetime import datetime
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import word_preprocessing as wpp
from textblob_de import TextBlobDE as TextBlob
import nltk

In [1]:
data = pd.read_csv("../data/daten_aijan_filtered.csv")
data.head()

,beitragid,beitragdt,titel,mediumid,anzahlzeichen,Abstract,Inhalt
0,21319059,2017-10-25 00:00:00.000,Über das Ziel hinausgeschossen,1,3178,NaN,"André Müller, amü\nWerde digitaler, sonst steh..."
1,21312273,2017-10-21 00:00:00.000,Gewagte Schätzungen für die SKA 2.0,2,3474,NaN,Gewagte Schätzungen für die SKA 2.0 ...
2,20946237,2017-05-24 00:00:00.000,"Der Spion, den wir liebten",4,7259,NaN,Jean-Claude Galli\nIn den letzten Jahren ist e...
3,21826660,2018-06-29 00:00:00.000,Endlich Hilfe gegen Migräne!,4,1041,NaN,Neue Spritze bald auf dem Markt 1 Million Schw...
4,21820335,2018-06-26 00:00:00.000,Swiss Life übernimmt deutsche Immobilienfirma,1,1362,NaN,(sda) · Die Swiss Life baut ihre Stellung am d...


In [2]:
# Datum bereinigen & redundante Zeilen löschen

data['datum'] = data['beitragdt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').date())
del data['beitragdt']
del data['Abstract']
data.head()

,beitragid,titel,mediumid,anzahlzeichen,Inhalt,datum
0,21319059,Über das Ziel hinausgeschossen,1,3178,"André Müller, amü\nWerde digitaler, sonst steh...",2017-10-25
1,21312273,Gewagte Schätzungen für die SKA 2.0,2,3474,Gewagte Schätzungen für die SKA 2.0 ...,2017-10-21
2,20946237,"Der Spion, den wir liebten",4,7259,Jean-Claude Galli\nIn den letzten Jahren ist e...,2017-05-24
3,21826660,Endlich Hilfe gegen Migräne!,4,1041,Neue Spritze bald auf dem Markt 1 Million Schw...,2018-06-29
4,21820335,Swiss Life übernimmt deutsche Immobilienfirma,1,1362,(sda) · Die Swiss Life baut ihre Stellung am d...,2018-06-26


In [20]:
# Headlines aufbereiten mit Klasse word_preprocessing
# testweise nur 100 erste Zeilen

preprocesser = wpp.headline_preprocessing()
data['titel_preprocessed'][:100] = preprocesser.preprocess_headlines(data['titel'][:100])
data

C:\Users\linus\AppData\Local\Temp\ipykernel_1156\1560632050.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['titel_preprocessed'][:100] = preprocesser.preprocess_headlines(data['titel'][:100])


,beitragid,titel,mediumid,anzahlzeichen,Inhalt,datum,Sentiment,titel_preprocessed
0,21319059,Über das Ziel hinausgeschossen,1,3178,"André Müller, amü\nWerde digitaler, sonst steh...",2017-10-25,0.150000,Ziel hinausgeschossen
1,21312273,Gewagte Schätzungen für die SKA 2.0,2,3474,Gewagte Schätzungen für die SKA 2.0 ...,2017-10-21,0.175714,Gewagt Schätzung SKA 20
2,20946237,"Der Spion, den wir liebten",4,7259,Jean-Claude Galli\nIn den letzten Jahren ist e...,2017-05-24,0.210423,Spion lieben
3,21826660,Endlich Hilfe gegen Migräne!,4,1041,Neue Spritze bald auf dem Markt 1 Million Schw...,2018-06-29,-0.200000,Hilfe Migran
4,21820335,Swiss Life übernimmt deutsche Immobilienfirma,1,1362,(sda) · Die Swiss Life baut ihre Stellung am d...,2018-06-26,0.231818,Swiss Life übernehmen deutsch Immobilienfirma
...,...,...,...,...,...,...,...,...
35892,22561096,kein Titel,946,1073,"Ein explosiver Vorwurf: «Was Roche tut, ist ei...",2019-02-09,NaN,NaN
35893,23150512,Schweizer Händler profitieren von Waldbränden ...,950,7467,In einem unscheinbaren Gebäude am Chemin de No...,2019-09-08,NaN,NaN
35894,23009642,Swiss Life begrüsst Pläne zu tieferem Umwandlu...,946,2113,"Die Vorschläge, welche die Sozialpartner und d...",2019-07-15,NaN,NaN
35895,23209865,«Die Reputation der Credit Suisse hat gelitten»,946,4153,Die Bespitzelungsaffäre bei der Credit Suisse ...,2019-10-01,NaN,NaN


In [14]:
def get_topic_sentiments(headlines):
    headline_blob = TextBlob(headlines)
    return headline_blob.sentiment.polarity

# Sentiment-Analyse
# testweise nur 100 erste Zeilen

data['Sentiment'] = data['Inhalt'][0:100].apply(lambda x: get_topic_sentiments(x))

In [15]:
data

,beitragid,titel,mediumid,anzahlzeichen,Inhalt,datum,Sentiment
0,21319059,Über das Ziel hinausgeschossen,1,3178,"André Müller, amü\nWerde digitaler, sonst steh...",2017-10-25,0.150000
1,21312273,Gewagte Schätzungen für die SKA 2.0,2,3474,Gewagte Schätzungen für die SKA 2.0 ...,2017-10-21,0.175714
2,20946237,"Der Spion, den wir liebten",4,7259,Jean-Claude Galli\nIn den letzten Jahren ist e...,2017-05-24,0.210423
3,21826660,Endlich Hilfe gegen Migräne!,4,1041,Neue Spritze bald auf dem Markt 1 Million Schw...,2018-06-29,-0.200000
4,21820335,Swiss Life übernimmt deutsche Immobilienfirma,1,1362,(sda) · Die Swiss Life baut ihre Stellung am d...,2018-06-26,0.231818
...,...,...,...,...,...,...,...
35892,22561096,kein Titel,946,1073,"Ein explosiver Vorwurf: «Was Roche tut, ist ei...",2019-02-09,NaN
35893,23150512,Schweizer Händler profitieren von Waldbränden ...,950,7467,In einem unscheinbaren Gebäude am Chemin de No...,2019-09-08,NaN
35894,23009642,Swiss Life begrüsst Pläne zu tieferem Umwandlu...,946,2113,"Die Vorschläge, welche die Sozialpartner und d...",2019-07-15,NaN
35895,23209865,«Die Reputation der Credit Suisse hat gelitten»,946,4153,Die Bespitzelungsaffäre bei der Credit Suisse ...,2019-10-01,NaN
